In [ ]:
import time
import datetime
import numpy as np
import pydicom
import cv2
import os
from pydicom.pixel_data_handlers.util import apply_voi_lut
import pandas as pd

In [ ]:
!mkdir ./downsized
!mkdir ./downsized/train
!mkdir ./downsized/test

In [ ]:
# get the current sizes of the images
BASE_PATH = "../input/vinbigdata-chest-xray-abnormalities-detection/"
ftrain = os.listdir(BASE_PATH + 'train')
ftest = os.listdir(BASE_PATH + 'test')

widths = []
heights = []

# # only examine the first 500 images to avoid this taking forever
# for i, file in enumerate(ftrain):
#     dicom = pydicom.read_file(BASE_PATH + "train/" + file)
#     data = apply_voi_lut(dicom.pixel_array, dicom)
#     widths.append(data.shape[1])
#     heights.append(data.shape[0])
    
#     if i > 500:
#         break

# print("Mean", np.mean(heights), np.mean(widths))
# print("Min", np.min(heights), np.min(widths))
# print("Max", np.max(heights), np.max(widths))

In [ ]:
# update the csv files
data = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
data.loc[data['class_name'] != 'No finding', 'x_min'] /= 2
data.loc[data['class_name'] != 'No finding', 'x_max'] /= 2
data.loc[data['class_name'] != 'No finding', 'y_min'] /= 2
data.loc[data['class_name'] != 'No finding', 'y_max'] /= 2
data.to_csv("downsized/train.csv")

In [ ]:

def read_xray(path, voi_lut = True, fix_monochrome = True, downscale_factor = 2):
    dicom = pydicom.read_file(path)

    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array

    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data

    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255.0).astype(np.uint8)
    new_shape = tuple([int(x / downscale_factor) for x in data.shape])
    data = cv2.resize(data, (new_shape[1], new_shape[0]))

    return data

for i in range(len(ftrain)):
    img = read_xray(BASE_PATH + 'train/'+ftrain[i])
    cv2.imwrite('./downsized/train/'+ftrain[i].replace('.dicom','.jpg'), img)
    if i % 500 == 0:
        print(datetime.datetime.now(), i,"out of", len(ftrain))

for i in range(len(ftest)):
    img = read_xray(BASE_PATH + 'test/'+ftest[i])
    cv2.imwrite('./downsized/test/'+ftest[i].replace('.dicom','.jpg'), img)  

## Write Data to COCO Format

Using code from https://www.kaggle.com/sreevishnudamodaran/vinbigdata-fusing-bboxes-coco-dataset

This code has been altered to include images with NO objects, whereas the original code only included images with objects.

In [ ]:
!pip install ensemble_boxes

In [ ]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
sns.set(rc={"font.size":9,"axes.titlesize":15,"axes.labelsize":9,
            "axes.titlepad":11, "axes.labelpad":9, "legend.fontsize":7,
            "legend.title_fontsize":7, 'axes.grid' : False})
import cv2
import json
import pandas as pd
import glob
import os.path as osp
from path import Path
import datetime
import numpy as np
from tqdm.auto import tqdm
import random
import shutil
from sklearn.model_selection import train_test_split

from ensemble_boxes import *
import warnings
from collections import Counter

In [ ]:
train_annotations = pd.read_csv("downsized/train.csv")
label2color = [[0,0,0], [59, 238, 119], [222, 21, 229], [94, 49, 164], [206, 221, 133], [117, 75, 3],
                 [210, 224, 119], [211, 176, 166], [63, 7, 197], [102, 65, 77], [194, 134, 175],
                 [209, 219, 50], [255, 44, 47], [89, 125, 149], [110, 27, 100]]

thickness = 3
imgs = []

In [ ]:
labels =  [
            "__ignore__",
            "Aortic_enlargement",
            "Atelectasis",
            "Calcification",
            "Cardiomegaly",
            "Consolidation",
            "ILD",
            "Infiltration",
            "Lung_Opacity",
            "Nodule/Mass",
            "Other_lesion",
            "Pleural_effusion",
            "Pleural_thickening",
            "Pneumothorax",
            "Pulmonary_fibrosis"
            ]

now = datetime.datetime.now()

data = dict(
    info=dict(
        description=None,
        url=None,
        version=None,
        year=now.year,
        contributor=None,
        date_created=now.strftime('%Y-%m-%d %H:%M:%S.%f'),
    ),
    licenses=[dict(
        url=None,
        id=0,
        name=None,
    )],
    images=[
        # license, url, file_name, height, width, date_captured, id
    ],
    type='instances',
    annotations=[
        # segmentation, area, iscrowd, image_id, bbox, category_id, id
    ],
    categories=[
        # supercategory, id, name
    ],
)

class_name_to_id = {}
for i, each_label in enumerate(labels):
    class_id = i - 1  # starts with -1
    class_name = each_label
    class_name_to_id[class_name] = class_id
    data['categories'].append(dict(
        supercategory=None,
        id=class_id,
        name=class_name,
    ))

In [ ]:
train_output_dir = "./data/train_images"
val_output_dir = "./data/val_images"

if not osp.exists(train_output_dir):
    os.makedirs(train_output_dir)
    print('Coco Train Image Directory:', train_output_dir)
    
if not osp.exists(val_output_dir):
    os.makedirs(val_output_dir)
    print('Coco Val Image Directory:', val_output_dir)
    
warnings.filterwarnings("ignore", category=UserWarning)    

In [ ]:
## Setting the output annotations json file path
train_out_file = './downsized/train_annotations.json'

data_train = data.copy()
data_train['images'] = []
data_train['annotations'] = []

iou_thr = 0.5
skip_box_thr = 0.0001
viz_images = []

for i, path in tqdm(enumerate(os.listdir("downsized/train"))):
    img_array  = cv2.imread(os.path.join("downsized", "train", path))
    image_basename = Path(path).stem
    
    
    ## Add Images to annotation
    data_train['images'].append(dict(
        license=0,
        url=None,
        file_name=os.path.join(path),
        height=img_array.shape[0],
        width=img_array.shape[1],
        date_captured=None,
        id=i
    ))
    
    img_annotations = train_annotations[train_annotations.image_id==image_basename]
    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()
    
    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []
    
    boxes_single = []
    labels_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    
    count_dict = Counter(img_annotations['class_id'].tolist())

    for cid in cls_ids:
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()
            
            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img_array.shape[0], img_array.shape[1], img_array.shape[0], img_array.shape[1])
            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())
            scores_list.append(np.ones(len(cls_list)).tolist())
            weights.append(1)
    
    ## Perform WBF
    boxes, scores, box_labels = weighted_boxes_fusion(boxes_list=boxes_list, scores_list=scores_list,
                                                  labels_list=labels_list, weights=weights,
                                                  iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes = boxes*(img_array.shape[0], img_array.shape[1], img_array.shape[0], img_array.shape[1])
    boxes = boxes.round(1).tolist()
    box_labels = box_labels.astype(int).tolist()
    boxes.extend(boxes_single)
    box_labels.extend(labels_single)
    
    img_after = img_array.copy()
    for box, label in zip(boxes, box_labels):
        x_min, y_min, x_max, y_max = (box[0], box[1], box[2], box[3])
        if str(x_min) != 'nan':
            bbox =[
                    round(x_min, 1),
                    round(y_min, 1),
                    round((x_max-x_min), 1),
                    round((y_max-y_min), 1)
                    ]
            area = (bbox[2] * bbox[3])
            data_train['annotations'].append(dict( id=len(data_train['annotations']), image_id=i,
                                                category_id=int(label), area=area, bbox=bbox,
                                                iscrowd=0))

with open(train_out_file, 'w') as f:
    json.dump(data_train, f, indent=4)

In [ ]:
## create COCO file for test data
now = datetime.datetime.now()

data = dict(
    info=dict(
        description=None,
        url=None,
        version=None,
        year=now.year,
        contributor=None,
        date_created=now.strftime('%Y-%m-%d %H:%M:%S.%f'),
    ),
    licenses=[dict(
        url=None,
        id=0,
        name=None,
    )],
    images=[
        # license, url, file_name, height, width, date_captured, id
    ],
    type='instances',
    annotations=[
        # segmentation, area, iscrowd, image_id, bbox, category_id, id
    ],
    categories=[
        # supercategory, id, name
    ],
)

class_name_to_id = {}
for i, each_label in enumerate(labels):
    class_id = i - 1  # starts with -1
    class_name = each_label
    class_name_to_id[class_name] = class_id
    data['categories'].append(dict(
        supercategory=None,
        id=class_id,
        name=class_name,
    ))

In [ ]:
for i, path in enumerate(os.listdir("downsized/test")):
    image = cv2.imread(os.path.join("downsized", "test", path))
    
    data['images'].append({"id": i, "file_name": path, "height": image.shape[0], "width": image.shape[1], "license": 0, "url": None})

with open("downsized/test_annotations.json", "w") as f:
    f.write(json.dumps(data))

In [ ]:
# zip the results
!zip -rq dataset.zip ./downsized/
!rm -rf downsized/